# Попытка 3

In [ ]:
import cv2 as cv2
import numpy as np
from matplotlib import pyplot as plt


PATH_TO_INPUT_IMAGES = 'images/examples/'

# IMAGE_NAME = '25.tif'
IMAGE_NAME = '26.tif'

IMAGE_PATH = PATH_TO_INPUT_IMAGES + IMAGE_NAME

In [ ]:
# Open image

img = cv2.imread(IMAGE_PATH)[3050:9500, 1000:8000]
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = np.uint8(gray)

plt.figure(figsize=(8, 8), dpi=60)
plt.imshow(X=gray)
plt.show()

In [ ]:
# Blur image

kernel_size = 7

blur_gray = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0) 

# plt.figure(figsize=(32, 32), dpi=60)
# plt.imshow(X=blur_gray)
# plt.show()
# cv2.imwrite('blur_gray.png', blur_gray)

In [ ]:
# Detecting edges

low_threshold = 140
high_threshold = 310

lines_edges = cv2.Canny(blur_gray, low_threshold, high_threshold)

# plt.figure(figsize=(32, 32), dpi=60)
# plt.imshow(X=lines_edges)
# plt.show()

In [ ]:
# Find corners of lines

lines_edges = np.float32(lines_edges)

# dst = cv2.cornerHarris(without_noise, 25, 31, 0.12)
dst = cv2.cornerHarris(lines_edges, 25, 15, 0.17)
dst = cv2.dilate(dst, None)

# img = cv2.imread(IMAGE_PATH)[3050:9500, 1000:8000]
img = cv2.imread(IMAGE_PATH)[3050:9500, 1000:8000]

crosses_points_matrix = dst > 0.12 * dst.max()
img[crosses_points_matrix] = [255, 0, 0]

plt.figure(figsize=(32, 32), dpi=100)
plt.imshow(X=img)
plt.show()

In [ ]:
colored_dots = crosses_points_matrix.nonzero()
print(f'Number of points of corners: {len(colored_dots[0])}')

In [ ]:
# Считаем расстояние между яркими точками

horisontal_values_matrix = (colored_dots[0] - colored_dots[0][:,np.newaxis]) ** 2  # (x_0 - x_1) ** 2
vertical_values_matrix = (colored_dots[1] - colored_dots[1][:,np.newaxis]) ** 2  # (y_0 - y_1) ** 2
distance_between_dots = np.sqrt(horisontal_values_matrix + vertical_values_matrix)


# Для того, что бы не было повторов, нижний треугольник заполняем большими значениями начиная с диагонали

distance_without_bottom_triangle = distance_between_dots + np.tri(*distance_between_dots.shape) * 100

In [ ]:
# Получаем пары близких друг к другу вершин

MAX_DISTANCE_BETWEEN_PIXELS = 30
closest_pixels = np.argwhere(distance_without_bottom_triangle < MAX_DISTANCE_BETWEEN_PIXELS)
len(closest_pixels)

In [ ]:
# Собираем кластеры

from typing import Set, List, Tuple

from pydantic import BaseModel

class Cluster(BaseModel):
    pixels_in_cluster: Set[int]
    close_pixels: List[Tuple[int, int]]

clusters: List[Cluster] = []

for pixels_pair in closest_pixels:
    was_inserted = False
    for existing_cluster in clusters:
        if pixels_pair[0] in existing_cluster.pixels_in_cluster:
            existing_cluster.pixels_in_cluster.update([pixels_pair[1]])
            existing_cluster.close_pixels.append(pixels_pair)
            was_inserted = True
            break
    if not was_inserted:
#         clusters.append([set((pixels_pair[0], pixels_pair[1])), [pixels_pair]])
        clusters.append(Cluster(
            pixels_in_cluster=set((pixels_pair[0], pixels_pair[1])),
            close_pixels=[tuple(pixels_pair)]
        ))

print(f'Было найдено: {len(clusters)} кластеров')

In [ ]:
# Восстанавливаем координаты точек кластера

clusters_coordinates: List[List[np.array]] = []

for cluster in clusters:
    cluster_coordinates: List[np.array] = []
    
    for point_in_cluster in cluster.pixels_in_cluster:
        cluster_coordinates.append([colored_dots[0][point_in_cluster], colored_dots[1][point_in_cluster]])
    
    clusters_coordinates.append(cluster_coordinates)

In [ ]:
# Рисуем кластеры
img = cv2.imread(IMAGE_PATH)[3050:9500, 1000:8000]

result = img.copy()
# result = img.copy() * 0

plt.figure(figsize=(32, 32), dpi=60)

cmap = plt.cm.get_cmap('hsv', len(clusters))
for number, cluster_coordinates in enumerate(clusters_coordinates):
    color = cmap(number)
    cluser_color = [255 * color[0], 255 * color[1], 255 * color[2]]

    for cluster_coordinate in cluster_coordinates:
        result[cluster_coordinate[0], cluster_coordinate[1]] = cluser_color

plt.imshow(X=result)
plt.show()
cv2.imwrite('clusters_on_lines_26.png', result)
# cv2.imwrite('clusters_black.png', result)

In [ ]:
# Рисуем центры кластеров и их координаты

cross_color = [0, 0, 255]
cluster_centers_x_y = []

for cluster_coordinates in clusters_coordinates:
    cluster_center = np.mean(cluster_coordinates, axis=0)
    cluster_center = [round(cluster_center[0]), round(cluster_center[1])]
    cluster_centers_x_y.append([cluster_center[1], cluster_center[0]])
    result[cluster_center[0], cluster_center[1]] = cross_color
    result[cluster_center[0], cluster_center[1]] = cross_color
    result[cluster_center[0]+1, cluster_center[1]+1] = cross_color
    result[cluster_center[0]-1, cluster_center[1]+1] = cross_color
    result[cluster_center[0]+1, cluster_center[1]-1] = cross_color
    result[cluster_center[0]-1, cluster_center[1]-1] = cross_color
    
    cv2.putText(
        result, 
        f'{cluster_center[1]}, {cluster_center[0]}', 
        (cluster_center[1], cluster_center[0]), 
        cv2.FONT_HERSHEY_PLAIN, 
        4,
        (0, 255, 0), 
        3,
        cv2.LINE_AA
    )

plt.figure(figsize=(32, 32), dpi=60)
plt.imshow(X=result)
plt.show()

cv2.imwrite('clusters_with_centers.png', result)

In [ ]:
# Clusters centers in order of columns
# For easier selecting extra centers

cluster_centers_sorted_by_x = sorted(
    [
        cluster_n
        for cluster_n in cluster_centers_x_y
    ], key=lambda x: x[0])
cluster_centers_sorted_by_x

In [ ]:
# Clusters centers in order of rows

cluster_centers_sorted_by_y = sorted(
    [
        cluster_n
        for cluster_n in cluster_centers_x_y
    ], key=lambda x: x[1])
# cluster_centers_sorted_by_y

In [ ]:
# Place extra centers in list 'elements_to_remove'

# elements_to_remove = [
#     [3556, 3729],
#     [2188, 4937],
#     [6214, 6209],
    
# ]
elements_to_remove = [
    [1596, 1312],
    [2056, 1313],
    [1585, 1786],
    [2047, 1782],
    [3894, 3812],
    [5960, 5386],
    [5949, 5857],
    [5727, 5623],
    [6184, 5625],
]

for element_to_remove in elements_to_remove:
    if element_to_remove in cluster_centers_sorted_by_y:
        cluster_centers_sorted_by_y.remove(element_to_remove)

print(f'Осталось {len(cluster_centers_sorted_by_y)} кластеров')

In [ ]:
# Set number of lines in one direction in grid
n_elements = 3

In [ ]:
# Build matrix of cluster centers

restored_matrix = [
    sorted(cluster_centers_sorted_by_y[i * n_elements:(i + 1) * n_elements], key=lambda x: x[0])
    for i in range(n_elements)
]
restored_matrix

In [ ]:
def get_distance_in_pixels(point_a, point_b):
    return [point_b[0] - point_a[0], point_b[1] - point_a[1]]

In [ ]:
# Build matrix of distance in pixels from top left corner

zero_point = restored_matrix[0][0]
relative_pixel_distances = [
    [
        get_distance_in_pixels(zero_point, restored_matrix[row][column]) 
        for column in range(n_elements)
    ]
    for row in range(n_elements)
]

for row in relative_pixel_distances:
     print(row)

In [ ]:
# Set dpi value of photo

dpi = 1200


dist_multiplier = 2.54 / dpi
dist_multiplier

In [ ]:
# Build matrix of distance in centimeters from top left corner

formating = "%.3f"
relative_distances = [
    [
        [formating%(column[0] * dist_multiplier), formating%(column[1] * dist_multiplier)]
        for column in row
    ]
    for row in relative_pixel_distances
]

for row in relative_distances:
     print(row)

# Поиск центров перекрестий №1

1. Кластарезуем светлые пиксели по расстоянию друг от друга
2. Расчитываем центры конечностей ребер, составляющих кресты
3. Определеяем центры крестов, как пересечение двух линий, образованных найденными центрами ребер

## Шаг 1: поиск кластеров крестов

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from random import randint

from pydantic import BaseModel

In [ ]:
# Дефим константы и всю хуйню

def convert_pixels_to_cm(pixels):
    return pixels / 2.54

PATH_TO_INPUT_IMAGES = 'images/examples'
IMAGE_NAME = 'example_crosses.jpg'
# IMAGE_NAME = 'example_upscale.jpg'

In [ ]:
# Открываем изображение

img = cv2.imread(filename=f'{PATH_TO_INPUT_IMAGES}/{IMAGE_NAME}')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

plt.figure(figsize=(8, 8), dpi=100)
plt.imshow(X=img)
plt.title(f'img')
plt.show()

In [ ]:
# Получаем координаты светлых точек

MINIMUM_BRIGHTNESS = 150
light_pixels = np.argwhere(gray[:,:] > MINIMUM_BRIGHTNESS)
len(light_pixels)

In [ ]:
# Я бы дополнительно нарисовал найденные точки тут
# light_pixels

In [ ]:
# Тупо что б не забыть, в каком порядке что идет

UPPER_BORDER_PIXEL = min(light_pixels[:,0])
LOWER_BORDER_PIXEL = max(light_pixels[:,0])

RIGHTES_BORDER_PIXEL = max(light_pixels[:,1])
LEFTES_BORDER_PIXEL = min(light_pixels[:,1])

LEFTES_BORDER_PIXEL, RIGHTES_BORDER_PIXEL, LOWER_BORDER_PIXEL, UPPER_BORDER_PIXEL

In [ ]:
# Считаем расстояние между яркими точками

horisontal_values_matrix = (light_pixels[:,0] - light_pixels[:,0][:,np.newaxis]) ** 2  # (x_0 - x_1) ** 2
vertical_values_matrix = (light_pixels[:,1] - light_pixels[:,1][:,np.newaxis]) ** 2  # (y_0 - y_1) ** 2
distance_between_dots = np.sqrt(horisontal_values_matrix + vertical_values_matrix)


# Для того, что бы не было повторов, нижний треугольник заполняем большими значениями начиная с диагонали

distance_without_bottom_triangle = distance_between_dots + np.tri(*distance_between_dots.shape) * 100

In [ ]:
# Получаем пары близких друг к другу вершин

MAX_DISTANCE_BETWEEN_PIXELS = 16
closest_pixels = np.argwhere(distance_without_bottom_triangle < MAX_DISTANCE_BETWEEN_PIXELS)
len(closest_pixels)

In [ ]:
# Собираем кластеры

from typing import Set, List, Tuple

class Cluster(BaseModel):
    pixels_in_cluster: Set[int]
    close_pixels: List[Tuple[int, int]]

clusters: List[Cluster] = []

for pixels_pair in closest_pixels:
    was_inserted = False
    for existing_cluster in clusters:
        if pixels_pair[0] in existing_cluster.pixels_in_cluster:
            existing_cluster.pixels_in_cluster.update([pixels_pair[1]])
            existing_cluster.close_pixels.append(pixels_pair)
            was_inserted = True
            break
    if not was_inserted:
#         clusters.append([set((pixels_pair[0], pixels_pair[1])), [pixels_pair]])
        clusters.append(Cluster(
            pixels_in_cluster=set((pixels_pair[0], pixels_pair[1])),
            close_pixels=[tuple(pixels_pair)]
        ))

print(f'Было найдено: {len(clusters)} кластеров')

In [ ]:
light_pixels

In [ ]:
# Восстанавливаем координаты точек кластера

clusters_coordinates: List[List[np.array]] = []

for cluster in clusters:
    cluster_coordinates: List[np.array] = []
    for point_in_cluster in cluster.pixels_in_cluster:
        cluster_coordinates.append(light_pixels[point_in_cluster])
    clusters_coordinates.append(cluster_coordinates)

In [ ]:
# Рисуем кластеры

result = img.copy()

for cluster_coordinates in clusters_coordinates:
#     cluser_color = [randint(100, 255), randint(100, 255), randint(100, 255)]
    cluser_color = [255, 0, 0]
    for cluster_coordinate in cluster_coordinates:
        result[cluster_coordinate[0], cluster_coordinate[1]] = cluser_color

plt.figure(figsize=(32, 32), dpi=60)
plt.imshow(X=result)
plt.title(f'img {len(img)}')
plt.show()

## Шаг 2: поиск центров крестов

In [ ]:
# Возьмем для начала крайние точки крестов и будем считать от них
        
def line(BaseModel):
    slope: float
    zero_point: float


centers_of_crosses = []

for cluster_coordinates in clusters_coordinates:
    cluster_row = np.array(cluster_coordinates)[:,1]
    lower_x_point = cluster_coordinates[np.argmin(cluster_row)]
    upper_x_point = cluster_coordinates[np.argmax(cluster_row)]
    x_points_delta = upper_x_point - lower_x_point

    horisonal_line_slope = x_points_delta[0] / x_points_delta[1]
    horisonal_line_shift = lower_x_point[0] - horisonal_line_slope * lower_x_point[1]
    
#     horisonal_y = horisonal_line_slope * horisonal_x - horisonal_line_shift
#     horisonal_y - horisonal_line_slope * horisonal_x + horisonal_line_shift = 0
    
    
    cluster_column = np.array(cluster_coordinates)[:,0]
    lower_y_point = cluster_coordinates[np.argmin(cluster_column)]
    upper_y_point = cluster_coordinates[np.argmax(cluster_column)]
    y_points_delta = upper_y_point - lower_y_point
    if y_points_delta[1] == 0:
        # Линия полностью вертикальна
        # Переделать алгоритм для корректного расчета. Наверно
        vertical_line_slope = 99999999
    else:
        vertical_line_slope = y_points_delta[0] / y_points_delta[1]
    vertical_line_shift = lower_y_point[0] - vertical_line_slope * lower_y_point[1]
    
#     horisonal_y = vertical_line_slope * vertical_y - vertical_line_shift
#     horisonal_y - vertical_line_slope * vertical_x + vertical_line_shift = 0
    
    
    x_center = (horisonal_line_shift - vertical_line_shift) / (vertical_line_slope - horisonal_line_slope)
    y_center = horisonal_line_slope * x_center + horisonal_line_shift
    
    centers_of_crosses.append([round(y_center), round(x_center)])

#     print(lower_x_point, upper_x_point, horisonal_line_slope, horisonal_line_shift, x_center)
#     print(lower_y_point, upper_y_point, vertical_line_slope, vertical_line_shift, y_center)
    
#     print()


In [ ]:
centers_of_crosses

In [ ]:
# Рисуем кластеры

for center_of_cross in centers_of_crosses:
#     cluser_color = [randint(100, 255), randint(100, 255), randint(100, 255)]
    cluser_color = [0, 0, 255]
    result[center_of_cross[0], center_of_cross[1]] = cluser_color
    result[center_of_cross[0]+1, center_of_cross[1]+1] = cluser_color
    result[center_of_cross[0]-1, center_of_cross[1]+1] = cluser_color
    result[center_of_cross[0]+1, center_of_cross[1]-1] = cluser_color
    result[center_of_cross[0]-1, center_of_cross[1]-1] = cluser_color

plt.figure(figsize=(32, 32), dpi=60)
plt.imshow(X=result)
plt.title(f'img {len(img)}')
plt.show()

## Шаг 3: упорядочивание крестов

## Шаг 4: рассчет расстояния между крестами

# Поиск центров перекрестий №2

1. Поиск пересечения линий

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from random import randint

import isect_segments_bentley_ottmann.poly_point_isect as bot

from pydantic import BaseModel

In [ ]:
PATH_TO_INPUT_IMAGES = 'images/examples'

In [ ]:
# img = cv2.imread(f'{PATH_TO_INPUT_IMAGES}/example.jpg')
# img = cv2.imread(f'{PATH_TO_INPUT_IMAGES}/sDQLM.png')
# img = cv2.imread(f'{PATH_TO_INPUT_IMAGES}/example_upscale.jpg')
# img = cv2.imread(f'{PATH_TO_INPUT_IMAGES}/25.tif')[3050:9500, 1000:8000]
img = cv2.imread(f'{PATH_TO_INPUT_IMAGES}/26.tif')[3050:9500, 1000:8000]
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

plt.figure(figsize=(8, 8), dpi=60)
plt.imshow(X=img)
plt.title(f'img {len(img)}')
# plt.show()

In [ ]:
kernel_size = 9

# blur_gray = cv2.blur(gray, (kernel_size, kernel_size), 0) * 5
blur_gray = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0) 

# plt.figure(figsize=(32, 32), dpi=600)
# plt.imshow(X=blur_gray)
# plt.show()
# cv2.imwrite('blur_gray.png', blur_gray)

In [ ]:
low_threshold = 10
high_threshold = 60

edges = cv2.Canny(blur_gray, low_threshold, high_threshold)

# plt.figure(figsize=(8, 8), dpi=60)
plt.imshow(X=edges)
plt.show()

# cv2.imwrite('canny.png', edges)

In [ ]:
def generate_points(rho, theta, threshold, min_line_length, max_line_gap):
    print(f'rho: {rho}, theta: {theta}, threshold: {threshold}, '
          f'min_line_length: {min_line_length}, max_line_gap: {max_line_gap}')
    theta = np.pi / theta
    line_image = np.copy(img) * 0  # creating a blank to draw lines on
    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(edges, rho, theta, threshold, np.array([]),
                        min_line_length, max_line_gap)
    points = []
    for line in lines:
        for x1, y1, x2, y2 in line:
            points.append(((x1, y1), (x2, y2)))
            cv2.line(img=line_image, pt1=(x1, y1), pt2=(x2, y2), color=(255, 0, 0), thickness=1)
    return points

#     plt.figure(figsize=(8, 8), dpi=120)
#     plt.imshow(X=line_image)
#     plt.show()

def generate_lines(img, line_image):
    lines_edges = cv2.addWeighted(img, 0.8, line_image, 1, 0)
    cv2.imwrite('line_parking.png', lines_edges)

#     plt.figure(figsize=(8, 8), dpi=60)
#     plt.imshow(X=lines_edges)
#     plt.show()
    return lines_edges

def plot_square(points, lines_edges, dpi =  120):
    intersections = bot.isect_segments(points)
    for inter in intersections:
        a, b = inter
        for i in range(3):
            for j in range(3):
                lines_edges[int(b) + i, int(a) + j] = [0, 255, 0]

    cv2.imwrite('result.png', lines_edges)
#     plt.figure(figsize=(32, 32), dpi=800)
#     plt.imshow(X=lines_edges)
#     plt.show()

In [ ]:
### rho, theta, threshold, min_line_length, max_line_gap

rho = 1  # distance resolution in pixels of the Hough grid
theta = np.pi / 180  # angular resolution in radians of the Hough grid
threshold = 15  # minimum number of votes (intersections in Hough grid cell)
min_line_length = 50  # minimum number of pixels making up a line
max_line_gap = 20  # maximum gap in pixels between connectable line segments
line_image = np.copy(img) * 0  # creating a blank to draw lines on

# for theta in range(10, 180*2, 10):
# for threshold in range(0, 40, 5): 
# for min_line_length in range(1, 40, 3):        
# for max_line_gap in range(1, 40, 3):

# points = generate_points(1, 180, 60, 50, 40)
# points = points + generate_points(1, 180, 40, 10, 40)
# points = points + generate_points(1, 180, 40, 10, 40)\
# points = generate_points(rho=1, theta=180, threshold=40, min_line_length=40, max_line_gap=40)
# points = generate_points(1, 90, 60, 10, 10) # + points 
# points = generate_points(2, 90, 60, 10, 10) # + points 
# points = points + generate_points(3, 180, 300, 10, 40)


# lines = generate_lines(img, line_image)
# plot_square(set(points), lines)

In [ ]:
lines

In [ ]:
print(f'rho: {rho}, theta: {theta}, threshold: {threshold}, '
      f'min_line_length: {min_line_length}, max_line_gap: {max_line_gap}')
theta = np.pi / theta
line_image = np.copy(img) * 0  # creating a blank to draw lines on
# Run Hough on edge detected image
# Output "lines" is an array containing endpoints of detected line segments
lines = cv2.HoughLinesP(edges, rho, theta, threshold, np.array([]),
                    min_line_length, max_line_gap)
points = []
for line in lines:
    for x1, y1, x2, y2 in line:
        points.append(((x1, y1), (x2, y2)))
        cv2.line(img=line_image, pt1=(x1, y1), pt2=(x2, y2), color=(255, 0, 0), thickness=1)


# plt.figure(figsize=(8, 8), dpi=120)
# plt.imshow(X=line_image)
# plt.show()


lines_edges = cv2.addWeighted(img, 0.8, line_image, 1, 0)
#cv2.imwrite('line_parking.png', lines_edges)

plt.figure(figsize=(8, 8), dpi=60)
plt.imshow(X=lines_edges)
plt.show()

intersections = bot.isect_segments(points)
for inter in intersections:
    a, b = inter
    for i in range(3):
        for j in range(3):
            lines_edges[int(b) + i, int(a) + j] = [0, 255, 0]

plt.figure(figsize=(8, 8), dpi=45)
plt.imshow(X=lines_edges)

plt.show()

In [ ]:
(edges, rho, theta, threshold, np.array([]),
                    min_line_length, max_line_gap)